# Tipos de BOTS

BOTs de DataFeed / Screeners

BOTs de Trading

* Bots de Colocación de grandes órdenes, para no alterar precio
* Bots de Colocación de órdenes, para obtener el mejor precio
* Bots de Market Making
* Bots de Arbitrajes en un mercado
* Bots de Arbitrajes entre mercados
* Bots de Arbitrajes estadísticos (en un mercado o entre mercados)
* Bots de Hedging
* Bots de Balanceo de carteras (optimización, riesgo, etc)
* Bots de ruteo para seguimiento de índices (fondos, etfs)
* Bots de scalping
* Bots de swing trading


BOTs estratégicos

* Bots de detección de otros bots

# API IOL

## Pedir Token por primera vez

In [2]:
from iol_user import *

iol_user = USER
iol_password = PASS

In [3]:
import requests

def pedirToken():
    url = 'https://api.invertironline.com/token'
    data = {"username" : iol_user, "password": iol_password, "grant_type":"password"}
    r = requests.post(url = url, data = data).json()
    return r

tk = pedirToken()

In [10]:
tk

{'access_token': '_agkGceQofNqUMCjEqf7U1NtUiu3RXxQlLG8I9tvGeaC2XWAgdnf6A-Z7r3eeZeV15iYZXttnAXa0hfR2qqXeh-bLc1gbMpqZMy_9-DNVak22O81zL1gEs0tpMqBbETnEvMOwbBqUtIpQVyM6rZGMewxYT3EPoc-CaEmjZCsH6QiePoZXGxrfiMFHUa1qC78HCqX3-VNxAO5V6NJ_urZHv6_gQQBIbynDazdgMQBpVUjkhO-PyrIsuuG_nGJRvW5R9wO5M6St-2MXZCg8DyuFY4plnNjsdpv3VbOEbUZWRi6aML8rqV8B3NAnts5LR2OqLFH9ZZhvKlpTCQW0u45rIr7cCjbZsUVPDjptQvqNLClyvbtps0DXl5az0DQ0yh_OAO-zXKLoGQzdutL1dSgFA',
 'token_type': 'bearer',
 'expires_in': 899,
 'refresh_token': 'k1JPUqiE9C6lwOFqdjoGqDy-4_bmbhs9jVP8qYkwFNTOrXh96EwR5jpPOMdKJfwctssthNXuus9xf42TJoxpLaQueEOFj8a0Qx6Yp0ULIAS0-HtQq2Wx4udHuZjh-lB2ljy6S4V-wRD1wa4icj7fruaYWIgtydcSHqGDGU3y5raFerUxh2mNKKlWBin_dN95GOs1gX4jyDgD8Oh8CaCYdm9uObmQNgFSVwrleO9oVykpoZHXVkVXywne6z_-bCsSm62d9ehVhdyN4lujqSzihXi-lQ814htY-CKHRmgWq2JDGZPtGU2ArcAU-dwEYwCQX7uj3b9L0LoDDMWyD-AVWbUTZ0xGkQ373ZI4Au3wnAQmxG7S-SqPV0Fjh-OLjt5YIIkNfDzHdjFF8rqGaYJjIg',
 '.issued': 'Wed, 11 Nov 2020 02:23:23 GMT',
 '.expires': 'Wed, 11 Nov 2020 02:38:23 GMT',
 '.refres

In [5]:
tk['access_token']

'_agkGceQofNqUMCjEqf7U1NtUiu3RXxQlLG8I9tvGeaC2XWAgdnf6A-Z7r3eeZeV15iYZXttnAXa0hfR2qqXeh-bLc1gbMpqZMy_9-DNVak22O81zL1gEs0tpMqBbETnEvMOwbBqUtIpQVyM6rZGMewxYT3EPoc-CaEmjZCsH6QiePoZXGxrfiMFHUa1qC78HCqX3-VNxAO5V6NJ_urZHv6_gQQBIbynDazdgMQBpVUjkhO-PyrIsuuG_nGJRvW5R9wO5M6St-2MXZCg8DyuFY4plnNjsdpv3VbOEbUZWRi6aML8rqV8B3NAnts5LR2OqLFH9ZZhvKlpTCQW0u45rIr7cCjbZsUVPDjptQvqNLClyvbtps0DXl5az0DQ0yh_OAO-zXKLoGQzdutL1dSgFA'

## Saber la exporacion del Token

In [6]:
tk['.expires']

'Wed, 11 Nov 2020 02:38:23 GMT'

In [11]:
import datetime as dt

In [12]:
dt.datetime.strptime(tk['.expires'],'%a, %d %b %Y %H:%M:%S GMT')

datetime.datetime(2020, 11, 11, 2, 38, 23)

In [13]:
dt.datetime.utcnow()

datetime.datetime(2020, 11, 11, 2, 25, 4, 486047)

In [15]:
dt.datetime.strptime(tk['.expires'],'%a, %d %b %Y %H:%M:%S GMT') - dt.datetime.utcnow()

datetime.timedelta(seconds=760, microseconds=254055)

Con ambas cosas puedo calcular la diferencia

In [16]:
def checkToken(tk):
    exp = dt.datetime.strptime(tk['.expires'],'%a, %d %b %Y %H:%M:%S GMT')
    ahora = dt.datetime.utcnow()
    tiempo = exp-ahora
    return tiempo

checkToken(tk)

datetime.timedelta(seconds=732, microseconds=751677)

Bien, el punto acá es que como la diferencia es positiva los días estan en 0, veamos:

In [22]:
checkToken(tk).days, checkToken(tk).seconds

(0, 613)

Como vemos en ese ejemplo me quedan aun o dias, 716 segundos, así que podría seguir usando mi access_token

Pero si se me expira el token no me quedan segundos negativos sino que queda days=-1 y los segundos comienzan cuenta regresiva desde 86400 que son 24 x 60 x 60 

## Actualizar Token (En caso que esté expirado)

Entonces armemos la función para que solo vuelva a pedir el access_token en caso de que el actual esté expirado:

In [21]:
def actualizarToken(tk):
    
    # Calculo el tiempo que le queda al tk
    import datetime as dt
    exp = dt.datetime.strptime(tk['.expires'],'%a, %d %b %Y %H:%M:%S GMT')
    ahora = dt.datetime.utcnow()
    tiempo = exp-ahora    
    dias = tiempo.days
    
    # Me fijo si está expirado
    if checkToken(tk).days == 0:
        tokenOk = tk
    else:
        tokenOk = pedirToken()
        
    # Devuelvo el token actualizado
    return tokenOk
    
actualizarToken(tk)

{'access_token': '_agkGceQofNqUMCjEqf7U1NtUiu3RXxQlLG8I9tvGeaC2XWAgdnf6A-Z7r3eeZeV15iYZXttnAXa0hfR2qqXeh-bLc1gbMpqZMy_9-DNVak22O81zL1gEs0tpMqBbETnEvMOwbBqUtIpQVyM6rZGMewxYT3EPoc-CaEmjZCsH6QiePoZXGxrfiMFHUa1qC78HCqX3-VNxAO5V6NJ_urZHv6_gQQBIbynDazdgMQBpVUjkhO-PyrIsuuG_nGJRvW5R9wO5M6St-2MXZCg8DyuFY4plnNjsdpv3VbOEbUZWRi6aML8rqV8B3NAnts5LR2OqLFH9ZZhvKlpTCQW0u45rIr7cCjbZsUVPDjptQvqNLClyvbtps0DXl5az0DQ0yh_OAO-zXKLoGQzdutL1dSgFA',
 'token_type': 'bearer',
 'expires_in': 899,
 'refresh_token': 'k1JPUqiE9C6lwOFqdjoGqDy-4_bmbhs9jVP8qYkwFNTOrXh96EwR5jpPOMdKJfwctssthNXuus9xf42TJoxpLaQueEOFj8a0Qx6Yp0ULIAS0-HtQq2Wx4udHuZjh-lB2ljy6S4V-wRD1wa4icj7fruaYWIgtydcSHqGDGU3y5raFerUxh2mNKKlWBin_dN95GOs1gX4jyDgD8Oh8CaCYdm9uObmQNgFSVwrleO9oVykpoZHXVkVXywne6z_-bCsSm62d9ehVhdyN4lujqSzihXi-lQ814htY-CKHRmgWq2JDGZPtGU2ArcAU-dwEYwCQX7uj3b9L0LoDDMWyD-AVWbUTZ0xGkQ373ZI4Au3wnAQmxG7S-SqPV0Fjh-OLjt5YIIkNfDzHdjFF8rqGaYJjIg',
 '.issued': 'Wed, 11 Nov 2020 02:23:23 GMT',
 '.expires': 'Wed, 11 Nov 2020 02:38:23 GMT',
 '.refres

## Uso de las funciones de Token

Ya con esas funciones puedo hacer lo siguiente:
    
* Si recien arranco a usar la API pongo tk=pedirToken() y ya me guardo en memoria mi token
* Luego una vez que use la API lo que hago es es usar tk = token(tk) (esto no va a volver a pedir un token a menos que el que tenga como parámetro esté expirado, en ese caso lo sobreescribirá, caso contrario quedará el mismo que estaba

Es decir solo cuando comience nuestro script usaremos:

In [66]:
tk = pedirToken()

y a partir de allí antes de cada request usaremos:

In [63]:
tk = actualizarToken(tk)

# MarketData

## Precios históricos

In [27]:
import pandas as pd

Empecemos con consultas de precios históricos, para ello tenemos los siguientes endpoints:

    * url_base = https://api.invertironline.com/api/v2/
    * endpoint = bCBA/Titulos/'+ticker+'/Cotizacion/seriehistorica/'+FROM+'/'+TO+'/sinAjustar
    * endpoint = bCBA/Titulos/'+ticker+'/Cotizacion/seriehistorica/'+FROM+'/'+TO+'/ajustada    

Como vemos los parámetros debemos pasarlos en la URL así que vamos a armar la función con ese criterio

La línea del resample es necesaria porque en una rueda hábil en horario de mercado me devuelve la serie de tiempo en dias pero la ultima rueda me devuelve todas las operaciones del día


In [25]:
def getHist(ticker,FROM,TO):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'bCBA/Titulos/'+ticker+'/Cotizacion/seriehistorica/'+FROM+'/'+TO+'/sinAjustar'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    # Acomodamos la tabla
    tabla = pd.DataFrame(data).set_index('fechaHora')
    tabla.index = pd.to_datetime(tabla.index)
    tabla = tabla.resample('d').last()       
    tabla = tabla.drop(['moneda','interesesAbiertos','puntas'], axis=1)

    return(tabla.dropna())

# Antes de pedir los precios actualizamos el token
tk = actualizarToken(tk)

# Pedimos los precios
data = getHist('GGAL',FROM='2015-01-01',TO='2019-05-05')

# mostramos las columnas de lo que trajimos
data.columns

Index(['ultimoPrecio', 'variacion', 'apertura', 'maximo', 'minimo',
       'tendencia', 'cierreAnterior', 'montoOperado', 'volumenNominal',
       'precioPromedio', 'precioAjuste', 'cantidadOperaciones'],
      dtype='object')

Y mostramos el primer dato todo lo que nos trae, no imprimo toda la tabla porque es inmensa

In [26]:
data.iloc[0]

ultimoPrecio                18.65
variacion                    0.81
apertura                    18.75
maximo                       19.1
minimo                      18.45
tendencia                    sube
cierreAnterior               18.5
montoOperado           7.7874e+06
volumenNominal             413569
precioPromedio             18.829
precioAjuste                    0
cantidadOperaciones           205
Name: 2015-01-02 00:00:00, dtype: object

## Precios históricos ajustados

In [28]:
def getHist(ticker,FROM,TO, Aj=False):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'    

    if Aj:
        endpoint = 'bCBA/Titulos/'+ticker+'/Cotizacion/seriehistorica/'+FROM+'/'+TO+'/ajustada'
    else:
        endpoint = 'bCBA/Titulos/'+ticker+'/Cotizacion/seriehistorica/'+FROM+'/'+TO+'/sinAjustar'

    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    # Acomodamos la tabla
    tabla = pd.DataFrame(data).set_index('fechaHora')
    tabla.index = pd.to_datetime(tabla.index)
    tabla = tabla.resample('d').last()       
    tabla = tabla.drop(['moneda','interesesAbiertos','puntas'], axis=1)

    return(tabla.dropna())

# Antes de pedir los precios actualizamos el token
tk = actualizarToken(tk)

# Pedimos los precios, esta vez ajustados
data = getHist('GGAL',FROM='2015-01-01',TO='2019-05-05', Aj=True)

# mostramos las columnas de lo que trajimos
data.iloc[0]

ultimoPrecio              18.4885
variacion                       0
apertura                  18.5877
maximo                    18.9346
minimo                    18.2903
tendencia                    sube
cierreAnterior                  0
montoOperado           7.7874e+06
volumenNominal             413569
precioPromedio                  0
precioAjuste                    0
cantidadOperaciones           205
Name: 2015-01-02 00:00:00, dtype: object

## Listado de FCIs

In [29]:
def getFCI():
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'Titulos/FCI'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    # Acomodamos la tabla
    tabla = pd.DataFrame(data)
    return(tabla)

tk = actualizarToken(tk) 
fcis = getFCI()

# Listemos las columnas para ver todo lo que trae
fcis.columns

Index(['variacion', 'ultimoOperado', 'horizonteInversion', 'rescate',
       'invierte', 'tipoFondo', 'avisoHorarioEjecucion',
       'tipoAdministradoraTituloFCI', 'fechaCorte', 'codigoBloomberg',
       'perfilInversor', 'informeMensual', 'reglamentoGestion', 'simbolo',
       'descripcion', 'pais', 'mercado', 'tipo', 'plazo', 'moneda'],
      dtype='object')

Agrupamos por tipo de fondo como para tener un pantallazo de la variedad:

In [30]:
fcis.groupby('tipoFondo').size()

tipoFondo
plazo_fijo_pesos        2
renta_fija_dolares      1
renta_fija_pesos        4
renta_mixta_dolares     1
renta_mixta_pesos       1
renta_variable_pesos    4
dtype: int64

Iteramos como para acceder a los links web de los reportes:

In [33]:
for idx, fci in fcis.iterrows():
    if fci.tipoFondo == 'renta_variable_pesos':
        print(fci.codigoBloomberg, '\n', fci.informeMensual,'\n\n')

PRTAVAB 
 https://s3.us-east-1.amazonaws.com/resources.invertironline.com/AssetManagement/FCI/Datasheets/SAM/Premier%20Renta%20Variable.pdf 


PCOMAGB 
 https://s3.us-east-1.amazonaws.com/resources.invertironline.com/AssetManagement/FCI/Datasheets/SAM/Premier%20Commodities.pdf 


CONRETO:AR 
 https://s3.us-east-1.amazonaws.com/resources.invertironline.com/AssetManagement/FCI/Datasheets/Adcap/Retorno%20Total.pdf 


COIOLAC:AR 
 https://s3.us-east-1.amazonaws.com/resources.invertironline.com/AssetManagement/FCI/Datasheets/Adcap/IOL%20Acciones.pdf 




In [34]:
fcis.loc[0]

variacion                                                                   3.22
ultimoOperado                                                             26.293
horizonteInversion                                                   Largo Plazo
rescate                                                                       t2
invierte                       Acciones cotizantes en la Bolsa de Comercio de...
tipoFondo                                                   renta_variable_pesos
avisoHorarioEjecucion          siguiente dia hábil, debido a que fue ingresad...
tipoAdministradoraTituloFCI                                          supervielle
fechaCorte                                                   1900-01-01T15:00:00
codigoBloomberg                                                          PRTAVAB
perfilInversor                 Para inversores agresivos que deseen un increm...
informeMensual                 https://s3.us-east-1.amazonaws.com/resources.i...
reglamentoGestion           

### FCIs Tipos de Fondos / Administradoras

También podemos consultar a la API los tipos de fondos y las administradoras con los siguientes endpoints:

In [35]:
def getFCI_TipoFondos():
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'Titulos/FCI/TipoFondos'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    return(data)

tk = actualizarToken(tk) 
getFCI_TipoFondos()

[{'identificador': 'plazo_fijo_pesos', 'nombre': 'Plazos Fijos Pesos'},
 {'identificador': 'renta_fija_pesos', 'nombre': 'Renta Fija Pesos'},
 {'identificador': 'renta_fija_dolares', 'nombre': 'Renta Fija Dólares'},
 {'identificador': 'renta_mixta_pesos', 'nombre': 'Renta Mixta Pesos'},
 {'identificador': 'renta_variable_pesos', 'nombre': 'Renta Variable Pesos'}]

In [36]:
def getFCI_Administradoras():
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'Titulos/FCI/Administradoras'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    return(data)

tk = actualizarToken(tk) 
getFCI_Administradoras()

[{'nombre': 'CONVEXITY',
  'descripcion': 'Administradora FCI para suscripcion y rescates'},
 {'nombre': 'SUPERVIELLE',
  'descripcion': 'Administradora FCI para suscripcion y rescates'},
 {'nombre': 'Allaria',
  'descripcion': 'Administradora FCI para suscripcion y rescates'}]

## Consulta de títulos

Tenemos un endpoint genérico que nos devuelve info general de un título o ticker

In [39]:
def titulo(ticker):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'bcba/Titulos/'+ticker
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    return(data)

tk = actualizarToken(tk) 
titulo('AL30D')

{'simbolo': 'AL30D',
 'descripcion': 'Bono Rep. Argentina Usd Step Up 2030',
 'pais': 'argentina',
 'mercado': 'bcba',
 'tipo': 'TITULOSPUBLICOS',
 'plazo': 't2',
 'moneda': 'dolar_Estadounidense'}

El mismo endpoint sirve para CEDEARs, Acciones, Bonos, FCIs, etc

Ejemplo con un CEDEAR

In [41]:
tk = actualizarToken(tk) 
titulo('AAPL')

{'simbolo': 'AAPL',
 'descripcion': 'Apple',
 'pais': 'argentina',
 'mercado': 'bcba',
 'tipo': 'CEDEARS',
 'plazo': 't2',
 'moneda': 'peso_Argentino'}

Ejemplo con una acción del panel general

In [42]:
tk = actualizarToken(tk) 
titulo('CELU')

{'simbolo': 'CELU',
 'descripcion': 'Celulosa',
 'pais': 'argentina',
 'mercado': 'bcba',
 'tipo': 'ACCIONES',
 'plazo': 't2',
 'moneda': 'peso_Argentino'}

Ejemplo con un FCI

In [43]:
tk = actualizarToken(tk) 
titulo('PRTAVAB')

{'simbolo': 'PRTAVAB',
 'descripcion': 'Premier Renta Variable - Clase B',
 'pais': 'argentina',
 'mercado': 'bcba',
 'tipo': 'FondoComundeInversion',
 'plazo': 't0',
 'moneda': 'peso_Argentino'}

## Instrumentos por pais

In [44]:
def instrumentos(pais):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = pais+'/Titulos/Cotizacion/Instrumentos'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    return(data)

tk = actualizarToken(tk) 
instrumentos('argentina')

[{'instrumento': 'Acciones', 'pais': 'argentina'},
 {'instrumento': 'Bonos', 'pais': 'argentina'},
 {'instrumento': 'Opciones', 'pais': 'argentina'},
 {'instrumento': 'Cauciones', 'pais': 'argentina'},
 {'instrumento': 'Futuros', 'pais': 'argentina'}]

Consultamos ahora los instrumentos disponibles para Estados Unidos

In [45]:
instrumentos('estados_unidos')

[{'instrumento': 'Acciones', 'pais': 'estados_Unidos'},
 {'instrumento': 'Bonos', 'pais': 'estados_Unidos'},
 {'instrumento': 'Etfs', 'pais': 'estados_Unidos'},
 {'instrumento': 'Monedas', 'pais': 'estados_Unidos'}]

## Paneles de un instrumento

In [46]:
def paneles(pais, instrumento):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = pais+'/Titulos/Cotizacion/Paneles/'+instrumento
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    return(data)

tk = actualizarToken(tk) 
paneles(pais='argentina', instrumento='Acciones')

[{'panel': 'Merval'},
 {'panel': 'Panel General'},
 {'panel': 'Merval 25'},
 {'panel': 'Merval Argentina'},
 {'panel': 'Burcap'},
 {'panel': 'CEDEARs'}]

Consultemos los paneles para acciones de EEUU

In [47]:
tk = actualizarToken(tk) 
paneles(pais='estados_unidos', instrumento='Acciones')

[{'panel': 'Dow Jones Industrial'},
 {'panel': 'Dow Jones Transportation'},
 {'panel': 'Dow Jones Utilities'},
 {'panel': 'Nasdaq 100'},
 {'panel': 'SP100'},
 {'panel': 'SP500'},
 {'panel': 'SP500 Value'},
 {'panel': 'SP500 Growth'},
 {'panel': 'SP400 MidCap'},
 {'panel': 'SP400 MidCap Value'},
 {'panel': 'S400 MidCap Growth'},
 {'panel': 'SP600 SmallCap'},
 {'panel': 'SP600 SmallCap Value'},
 {'panel': 'SP600 SmallCap Growth'},
 {'panel': 'SP500 Dividendos'},
 {'panel': 'ADRs'}]

In [48]:
tk = actualizarToken(tk) 
paneles(pais='estados_unidos', instrumento='Etfs')

[{'panel': 'Todos'},
 {'panel': 'Acciones Desarrollados'},
 {'panel': 'Acciones Emergentes'},
 {'panel': 'Acciones Financieras'},
 {'panel': 'Acciones Energéticas'},
 {'panel': 'Acciones Tecnológicas'},
 {'panel': 'Acciones Consumo Básico'},
 {'panel': 'Acciones Consumo Durable'},
 {'panel': 'Acciones Salud'},
 {'panel': 'Acciones Industriales'},
 {'panel': 'Acciones Materiales'},
 {'panel': 'Acciones Servicios Básicos'},
 {'panel': 'Bonos'},
 {'panel': 'Commodities'},
 {'panel': 'Monedas'},
 {'panel': 'Inmuebles'},
 {'panel': 'Estrategias Avanzadas'},
 {'panel': 'Apalancados Alcistas'},
 {'panel': 'Apalancados Bajistas'}]

In [49]:
tk = actualizarToken(tk) 
paneles(pais='argentina', instrumento='Opciones')

[{'panel': 'De Acciones'},
 {'panel': 'De Bonos'},
 {'panel': 'De Cedears'},
 {'panel': 'In-the-Money'},
 {'panel': 'Out-the-Money'},
 {'panel': 'Calls'},
 {'panel': 'Puts'}]

## Paneles de precios

Este es el endpoint mas poderoso de la API

In [50]:
def panel(instrumento, panel,pais):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'Cotizaciones/'+instrumento+'/'+panel+'/'+pais
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()['titulos']

    tabla = pd.DataFrame(data).set_index('simbolo')
    return(tabla)


tk = actualizarToken(tk) 
p = panel(instrumento='Acciones', panel='SP500', pais='estados_unidos')
p.columns

Index(['puntas', 'ultimoPrecio', 'variacionPorcentual', 'apertura', 'maximo',
       'minimo', 'ultimoCierre', 'volumen', 'cantidadOperaciones', 'fecha',
       'tipoOpcion', 'precioEjercicio', 'fechaVencimiento', 'mercado',
       'moneda'],
      dtype='object')

Veamos una acción cualquiera del panel:

In [51]:
p.loc['AAPL']

puntas                 {'cantidadCompra': 1.0, 'precioCompra': 116.02...
ultimoPrecio                                                         116
variacionPorcentual                                                -0.28
apertura                                                          115.55
maximo                                                            117.59
minimo                                                            114.13
ultimoCierre                                                      116.32
volumen                                                      2.81124e+08
cantidadOperaciones                                                    0
fecha                                                2020-11-10T19:57:03
tipoOpcion                                                          None
precioEjercicio                                                     None
fechaVencimiento                                                    None
mercado                                            

O bien sus puntas de compra y venta:

In [52]:
p.loc['AAPL']['puntas']

{'cantidadCompra': 1.0,
 'precioCompra': 116.02,
 'precioVenta': 116.12,
 'cantidadVenta': 100.0}

## Precio de UN instrumento

Ingresando el mercado y el ticker podemos obtener el precio de dicho ticker

In [53]:
def precio(mercado, ticker):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = mercado+'/Titulos/'+ticker+'/Cotizacion'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    return data

tk = actualizarToken(tk) 
data = precio('bcba','GGAL')
data

{'ultimoPrecio': 128.75,
 'variacion': 5.27,
 'apertura': 124.0,
 'maximo': 132.3,
 'minimo': 122.3,
 'fechaHora': '2020-11-10T17:14:06.640721-03:00',
 'tendencia': 'sube',
 'cierreAnterior': 122.3,
 'montoOperado': 729048703.1499981,
 'volumenNominal': 5685205,
 'precioPromedio': 0.0,
 'moneda': 'peso_Argentino',
 'precioAjuste': 0.0,
 'interesesAbiertos': 0.0,
 'puntas': [{'cantidadCompra': 17.0,
   'precioCompra': 126.0,
   'precioVenta': 130.0,
   'cantidadVenta': 476.0},
  {'cantidadCompra': 564.0,
   'precioCompra': 127.5,
   'precioVenta': 129.75,
   'cantidadVenta': 2558.0},
  {'cantidadCompra': 180.0,
   'precioCompra': 127.55,
   'precioVenta': 129.8,
   'cantidadVenta': 12877.0},
  {'cantidadCompra': 34.0,
   'precioCompra': 125.5,
   'precioVenta': 129.85,
   'cantidadVenta': 385.0},
  {'cantidadCompra': 1000.0,
   'precioCompra': 127.6,
   'precioVenta': 129.9,
   'cantidadVenta': 210.0}],
 'cantidadOperaciones': 7742}

### Precios RealTime de acciones en EEUU

Probamos con GGAL, tanto en nyse como en nasdaq y obtenemos su precio en ambos mercados

In [54]:
tk = actualizarToken(tk) 
data = precio('nyse','GGAL')
data

{'ultimoPrecio': 8.8,
 'variacion': 8.77626699629172,
 'apertura': 8.26,
 'maximo': 8.9,
 'minimo': 8.02,
 'fechaHora': '2020-11-10T17:58:13',
 'tendencia': 'sube',
 'cierreAnterior': 8.09,
 'montoOperado': 37092.0,
 'volumenNominal': 0,
 'precioPromedio': 0.0,
 'moneda': 'dolar_Estadounidense',
 'precioAjuste': 0.0,
 'interesesAbiertos': 0.0,
 'puntas': [{'cantidadCompra': 100.0,
   'precioCompra': 8.11,
   'precioVenta': 9.46,
   'cantidadVenta': 100.0}],
 'cantidadOperaciones': 0}

In [55]:
tk = actualizarToken(tk) 
data = precio('nasdaq','GGAL')
data

{'ultimoPrecio': 8.8,
 'variacion': 8.77626699629172,
 'apertura': 8.26,
 'maximo': 8.9,
 'minimo': 8.02,
 'fechaHora': '2020-11-10T17:58:13',
 'tendencia': 'sube',
 'cierreAnterior': 8.09,
 'montoOperado': 37092.0,
 'volumenNominal': 0,
 'precioPromedio': 0.0,
 'moneda': 'dolar_Estadounidense',
 'precioAjuste': 0.0,
 'interesesAbiertos': 0.0,
 'puntas': [{'cantidadCompra': 100.0,
   'precioCompra': 8.11,
   'precioVenta': 9.46,
   'cantidadVenta': 100.0}],
 'cantidadOperaciones': 0}

### Precios Real Time de una opción:

In [56]:
tk = actualizarToken(tk) 
data = precio('bcba','GFGC117.AG')
data

{'ultimoPrecio': 20.343,
 'variacion': 0.0,
 'apertura': 18.0,
 'maximo': 21.5,
 'minimo': 18.0,
 'fechaHora': '2020-11-04T03:00:15.113',
 'tendencia': 'mantiene',
 'cierreAnterior': 20.343,
 'montoOperado': 0.0,
 'volumenNominal': 0,
 'precioPromedio': 0.0,
 'moneda': 'peso_Argentino',
 'precioAjuste': 0.0,
 'interesesAbiertos': 0.0,
 'puntas': [{'cantidadCompra': 0.0,
   'precioCompra': 0.0,
   'precioVenta': 0.0,
   'cantidadVenta': 0.0}],
 'cantidadOperaciones': 0}

### Precios Real Time de bonos

In [57]:
tk = actualizarToken(tk) 
data = precio('bcba','AL30D')
data

{'ultimoPrecio': 38.4,
 'variacion': 0.13,
 'apertura': 38.25,
 'maximo': 38.49,
 'minimo': 38.1,
 'fechaHora': '2020-11-10T17:00:50.1987307-03:00',
 'tendencia': 'sube',
 'cierreAnterior': 38.35,
 'montoOperado': 7624949.04,
 'volumenNominal': 19927985,
 'precioPromedio': 0.0,
 'moneda': 'dolar_Estadounidense',
 'precioAjuste': 0.0,
 'interesesAbiertos': 0.0,
 'puntas': [{'cantidadCompra': 1147.0,
   'precioCompra': 38.0,
   'precioVenta': 38.7,
   'cantidadVenta': 2000.0},
  {'cantidadCompra': 4000.0,
   'precioCompra': 25.0,
   'precioVenta': 38.97,
   'cantidadVenta': 154.0},
  {'cantidadCompra': 1.0,
   'precioCompra': 5.0,
   'precioVenta': 39.0,
   'cantidadVenta': 23391.0},
  {'cantidadCompra': 0.0,
   'precioCompra': 0.0,
   'precioVenta': 39.45,
   'cantidadVenta': 1000.0},
  {'cantidadCompra': 0.0,
   'precioCompra': 0.0,
   'precioVenta': 39.5,
   'cantidadVenta': 110.0}],
 'cantidadOperaciones': 3321}

### Precios de FCIs

Con un FCI también funciona, aunque al no tener mercado secundario, solo tenemos precios EOD (fin del dia), que es valor de cuotaparte en realidad al cierre

In [58]:
tk = actualizarToken(tk) 
data = precio('bcba','PRTAVAB')
data

{'ultimoPrecio': 25.472421,
 'variacion': 2.86,
 'apertura': 0.0,
 'maximo': 0.0,
 'minimo': 0.0,
 'fechaHora': '2020-11-09T18:00:00',
 'tendencia': 'sube',
 'cierreAnterior': 0.0,
 'montoOperado': 0.0,
 'volumenNominal': 0,
 'precioPromedio': 0.0,
 'moneda': 'peso_Argentino',
 'precioAjuste': 0.0,
 'interesesAbiertos': 0.0,
 'puntas': [{'cantidadCompra': 0.0,
   'precioCompra': 0.0,
   'precioVenta': 0.0,
   'cantidadVenta': 0.0}],
 'cantidadOperaciones': 0}

### También funciona con un ETF

In [59]:
tk = actualizarToken(tk) 
data = precio('nyse','EEM')
data

{'ultimoPrecio': 47.73,
 'variacion': -1.1,
 'apertura': 48.07,
 'maximo': 48.19,
 'minimo': 47.5,
 'fechaHora': '2020-11-10T20:20:17',
 'tendencia': 'baja',
 'cierreAnterior': 48.26,
 'montoOperado': 2426115.9,
 'volumenNominal': 0,
 'precioPromedio': 0.0,
 'moneda': 'dolar_Estadounidense',
 'precioAjuste': 0.0,
 'interesesAbiertos': 0.0,
 'puntas': [{'cantidadCompra': 100.0,
   'precioCompra': 47.73,
   'precioVenta': 47.85,
   'cantidadVenta': 800.0}],
 'cantidadOperaciones': 0}

## Opciones de un determinado subyacente

In [60]:
def opcionesDe(ticker):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'bcba/Titulos/'+ticker+'/Opciones'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    opciones = []
    for i in range(len(data)):
        opcion = data[i]['cotizacion']
        opcion['simbolo'] = data[i]['simbolo']
        opcion['tipo'] = data[i]['tipoOpcion']
        opcion['vencimiento'] = data[i]['fechaVencimiento']
        opcion['descripcion'] = data[i]['descripcion']
        opciones.append(opcion)
    tabla = pd.DataFrame(opciones).set_index('simbolo')
    return(tabla)

tk = actualizarToken(tk) 
data = opcionesDe('GGAL')
data.columns

Index(['ultimoPrecio', 'variacion', 'apertura', 'maximo', 'minimo',
       'fechaHora', 'tendencia', 'cierreAnterior', 'montoOperado',
       'volumenNominal', 'precioPromedio', 'moneda', 'precioAjuste',
       'interesesAbiertos', 'puntas', 'cantidadOperaciones', 'tipo',
       'vencimiento', 'descripcion'],
      dtype='object')

Como ven en las columnas tenemos de todo, veamos por ejemplo como ver la cantidad de contratos tanto de calls como de puts por cada vencimiento:

In [61]:
data.groupby(['tipo','vencimiento']).size()

tipo  vencimiento        
Call  2020-11-20T00:00:00     3
      2020-12-18T00:00:00    27
      2021-02-19T00:00:00    14
      2021-04-16T00:00:00     1
Put   2020-12-18T00:00:00    26
      2021-02-19T00:00:00     4
      2021-04-16T00:00:00     1
dtype: int64

Asimismo puedo traer data concreta de una opción determinada:

In [63]:
data.index.values

array(['GFGC123.NO', 'GFGC135.NO', 'GFGC140.NO', 'GFGC49808D',
       'GFGC52808D', 'GFGC63.0DI', 'GFGV63.0DI', 'GFGV75.0DI',
       'GFGV79808D', 'GFGV85808D', 'GFGC88808D', 'GFGV88808D',
       'GFGV93.0DI', 'GFGV96.0DI', 'GFGC96.0DI', 'GFGC97808D',
       'GFGV97808D', 'GFGC10081D', 'GFGV10081D', 'GFGC105.DI',
       'GFGV105.DI', 'GFGC108.DI', 'GFGV108.DI', 'GFGC10981D',
       'GFGV10981D', 'GFGC11281D', 'GFGV11281D', 'GFGC11581D',
       'GFGV11581D', 'GFGC11881D', 'GFGV11881D', 'GFGC12181D',
       'GFGV12181D', 'GFGC12481D', 'GFGV12481D', 'GFGC12781D',
       'GFGV12781D', 'GFGV13081D', 'GFGC13081D', 'GFGC13381D',
       'GFGV13381D', 'GFGC13881D', 'GFGV13881D', 'GFGC145.DI',
       'GFGV145.DI', 'GFGV14881D', 'GFGC14881D', 'GFGC155.DI',
       'GFGC15881D', 'GFGV15881D', 'GFGC16381D', 'GFGV16381D',
       'GFGC170.DI', 'GFGC175.DI', 'GFGV19881D', 'GFGC19881D',
       'GFGV82808F', 'GFGC87.0FE', 'GFGC96.0FE', 'GFGC97808F',
       'GFGV97808F', 'GFGC114.FE', 'GFGV11881F', 'GFGC1

In [66]:
data.loc['GFGC13381D']

ultimoPrecio                                               6.05
variacion                                                 27.87
apertura                                                      0
maximo                                                        0
minimo                                                        0
fechaHora                                   0001-01-01T00:00:00
tendencia                                                  sube
cierreAnterior                                                0
montoOperado                                                  0
volumenNominal                                                0
precioPromedio                                                0
moneda                                                        0
precioAjuste                                                  0
interesesAbiertos                                             0
puntas                                                     None
cantidadOperaciones                     

# Endpoints de Cuenta

## Estado de Cuenta

In [67]:
def cuenta():
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'estadocuenta'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    return(data)

tk = actualizarToken(tk) 
data = cuenta()
data.keys()

dict_keys(['cuentas', 'estadisticas', 'totalEnPesos'])

In [72]:
data['cuentas'][0]

{'numero': '200487',
 'tipo': 'inversion_Argentina_Pesos',
 'moneda': 'peso_Argentino',
 'disponible': 1016.98,
 'comprometido': 0.0,
 'saldo': 1016.98,
 'titulosValorizados': 0.0,
 'total': 1016.98,
 'margenDescubierto': 0.0,
 'saldos': [{'liquidacion': 'inmediato',
   'saldo': 1016.98,
   'comprometido': 0.0,
   'disponible': 1016.98,
   'disponibleOperar': 1016.98},
  {'liquidacion': 'hrs24',
   'saldo': 0.0,
   'comprometido': 0.0,
   'disponible': 0.0,
   'disponibleOperar': 1016.98},
  {'liquidacion': 'hrs48',
   'saldo': 0.0,
   'comprometido': 0.0,
   'disponible': 0.0,
   'disponibleOperar': 1016.98},
  {'liquidacion': 'hrs72',
   'saldo': 0.0,
   'comprometido': 0.0,
   'disponible': 0.0,
   'disponibleOperar': 1016.98},
  {'liquidacion': 'masHrs72',
   'saldo': 0.0,
   'comprometido': 0.0,
   'disponible': 0.0,
   'disponibleOperar': 0.0}],
 'estado': 'operable'}

In [75]:
data['estadisticas']

[{'descripcion': 'Anterior', 'cantidad': 0, 'volumen': 0.0},
 {'descripcion': 'Actual', 'cantidad': 0, 'volumen': 0.0}]

## Portafolio

In [78]:
def portafolio(pais):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'portafolio/'+pais
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    
    return(data)

tk = actualizarToken(tk) 
data = portafolio('Argentina')
data

{'pais': 'argentina', 'activos': []}

## Operaciones vigentes

In [91]:
def operaciones():
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'operaciones/'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    df = pd.DataFrame(data)
    return df

tk = actualizarToken(tk) 
data = operaciones()
data.columns

ValueError: If using all scalar values, you must pass an index

# Endpoints de Ruteo

## Operar: Compra

In [82]:
def comprar(ticker, q, precio, plazo ):
    import datetime as dt
    vigencia = dt.datetime.now() + dt.timedelta(days=1)
    vigencia_str = dt.datetime.strftime(vigencia, '%Y-%m-%d')
    params = {
      "mercado": "bCBA",
      "simbolo": ticker,
      "cantidad": q,
      "precio": precio,
      "plazo": plazo,
      "validez": vigencia_str
    }
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'operar/comprar/'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.post(url = url, headers = headers, json = params).json()
    return data

¿Que voy a ahacer antes de comprar?

In [87]:
tk = actualizarToken(tk) 
data = precio('bcba','COME')
data

{'ultimoPrecio': 2.8,
 'variacion': 1.81,
 'apertura': 2.78,
 'maximo': 2.82,
 'minimo': 2.7,
 'fechaHora': '2020-11-10T17:00:06.2643252-03:00',
 'tendencia': 'sube',
 'cierreAnterior': 2.75,
 'montoOperado': 9213685.75,
 'volumenNominal': 3317536,
 'precioPromedio': 0.0,
 'moneda': 'peso_Argentino',
 'precioAjuste': 0.0,
 'interesesAbiertos': 0.0,
 'puntas': [{'cantidadCompra': 270.0,
   'precioCompra': 2.75,
   'precioVenta': 2.85,
   'cantidadVenta': 4807.0},
  {'cantidadCompra': 10100.0,
   'precioCompra': 2.74,
   'precioVenta': 2.87,
   'cantidadVenta': 13530.0},
  {'cantidadCompra': 20294.0,
   'precioCompra': 2.73,
   'precioVenta': 2.9,
   'cantidadVenta': 14247.0},
  {'cantidadCompra': 21963.0,
   'precioCompra': 2.72,
   'precioVenta': 2.98,
   'cantidadVenta': 25439.0},
  {'cantidadCompra': 50117.0,
   'precioCompra': 2.71,
   'precioVenta': 3.0,
   'cantidadVenta': 904.0}],
 'cantidadOperaciones': 892}

In [88]:
data['puntas'][0]

{'cantidadCompra': 270.0,
 'precioCompra': 2.75,
 'precioVenta': 2.85,
 'cantidadVenta': 4807.0}

In [89]:
pta_bid = data['puntas'][0]['precioCompra']

In [90]:
tk = actualizarToken(tk) 
comprar(ticker='COME', q=1, precio = pta_bid+0.01, plazo='t2')

{'message': 'En estos momentos estamos trabajando en la actualización de la información solicitada.'}

## Ver estado de Operación

In [83]:
def operacion(numero):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'operaciones/'+str(numero)
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.get(url = url, headers = headers).json()
    return data

In [ ]:
tk = actualizarToken(tk) 
data = operacion(24166668)
data

## Operar Venta

In [84]:
def vender(ticker, q, precio, plazo ):
    import datetime as dt
    vigencia = dt.datetime.now() + dt.timedelta(days=1)
    vigencia_str = dt.datetime.strftime(vigencia, '%Y-%m-%d')
    params = {
      "mercado": "bCBA",
      "simbolo": ticker,
      "cantidad": q,
      "precio": precio,
      "plazo": plazo,
      "validez": vigencia_str
    }
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'operar/vender/'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.post(url = url, headers = headers, json = params).json()
    return data


In [ ]:
tk = actualizarToken(tk) 
vender(ticker='COME', q=10, precio=2.4, plazo='t2')

In [333]:
tk = actualizarToken(tk) 
data = operaciones()

cols = ['numero','tipo','simbolo','cantidad','precio','cantidadOperada']
data.loc[data.estado=='iniciada', cols]

,numero,tipo,simbolo,cantidad,precio,cantidadOperada
0,24166670,Venta,COME,10.0,2.4,NaN
1,24166668,Compra,ALUA,1.0,40.1,NaN


## Operar, Cancelar Orden

In [85]:
def cancelar(numero):
    
    # Configuración del llamado a la API
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'operaciones/'+str(numero)
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    
    # Llamado a la API
    data = requests.delete(url = url, headers = headers).json()
    return data


In [ ]:
tk = actualizarToken(tk) 

# Cancelamos primero la compra de ALUA
data = cancelar(24166668)
data

In [335]:
tk = actualizarToken(tk) 
data = operaciones()

cols = ['numero','tipo','simbolo','cantidad','precio','cantidadOperada']
data.loc[data.estado=='iniciada', cols]

,numero,tipo,simbolo,cantidad,precio,cantidadOperada
0,24166670,Venta,COME,10.0,2.4,NaN
